In [ ]:
#import urllib2
from io import StringIO
import os
import xmltodict
import pandas as pd
import geopandas as gpd
import windrose
from windrose import WindroseAxes
import matplotlib.cm as cm

from datetime import datetime 
import statsmodels.api as sm
import time

from scipy.stats import linregress
import matplotlib.pyplot as plt
import numpy as np
import sys
import requests
import glob

from pyproj import CRS
from pyproj import Transformer
from shapely.geometry import Point

import rasterstats as rs
import rasterio

import requests

from rasterio.plot import show
from rasterio.mask import mask
from rasterio.plot import show_hist
from shapely.geometry import box
from rasterstats import zonal_stats
from windrose import WindroseAxes
#import pycrs
#import earthpy as et
#import earthpy.plot as ep

from scipy.optimize import curve_fit

from sklearn import mixture
import itertools
from scipy import linalg
import matplotlib as mpl

import cartopy


#import pymannkendall as mk
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 15, 10

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location("dbconnect", "G:/My Drive/Python/dbconnect.py")
dbconnect = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dbconnect)

ucc_key = dbconnect.get_ucc_api()

In [ ]:
import geopandas
import contextily as ctx
latitude = 39.014255
longitude = -110.166383
source = 'UCC'
url = f"https://climate.usu.edu/API/api.php/v2/key={ucc_key}/station_search/source={source}/radius=15/lat={latitude}/long={longitude}"
print(url)
resp = requests.get(url)


In [ ]:
import json
try:
    response_ob = resp.json()
except:
    response_ob = json.loads(resp.text[resp.text.find('\n\n')+2:])

In [ ]:
response_ob 

In [ ]:
df = pd.DataFrame(response_ob['payload'])
ghcn_stations = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.longitude, df.latitude, crs='EPSG:4269')).to_crs(epsg=3857)
print(ghcn_stations)
ax = ghcn_stations.plot(figsize=(14, 14), column='name',legend=False)
ax.axis((longitude-0.1,longitude+0.1, latitude-0.1, latitude+0.1))
#ctx.add_basemap(ax)

In [ ]:
https://climate.usu.edu/API/api.php/v3/key=/station_search/source=SCAN/state=UT

In [ ]:
stations

In [ ]:
stations = df.set_index(['station_id'])

out_format = 'json'
start_date = '2020-10-01T00:00:00'
end_date = '2024-07-12T00:00:00'
freq = 'get_daily' #get_hourly
 
newbegdate = pd.to_datetime(start_date)
newenddate = newbegdate + pd.Timedelta(29*6,'D')

hed = {}
dat = {}

for stat in stations.index.unique():
    if stat == '124033':
        scandat = {}
        source = stations.loc[stat,'network']
        while newbegdate <= datetime.now():
            if freq == 'get_daily':
                begstr = newbegdate.strftime("%Y-%m-%d")
                endstr = newenddate.strftime("%Y-%m-%d")
            else:
                begstr = newbegdate.strftime("%Y-%m-%dT%H:%M:%S")
                endstr = newenddate.strftime("%Y-%m-%dT%H:%M:%S")

            url = f"https://climate.usu.edu/API/api.php/v2/key={ucc_key}/station_search/network={source}/station_id={stat}/{freq}/start_date={begstr}/end_date={endstr}/units=m"
            print(url)
            response_ob = requests.get(url).json()
            newbegdate += pd.Timedelta(29*6,'D')
            newenddate += pd.Timedelta(29*6,'D') 
            scandat[endstr] = pd.DataFrame(response_ob['payload'])

            time.sleep(1)
        dat[stations.loc[stat,"name"]] = pd.concat(scandat)


In [ ]:
stat = '124033'
dat[stations.loc[stat,"name"]] = pd.concat(scandat)

In [ ]:
dat[stations.loc[stat,"name"][0]] = pd.concat(scandat)

In [ ]:
dat = pd.read_csv("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/Data_Downloads/BSF/previous_data/BFLAT.2021-09-14.csv",
                 skiprows=[0,1,2,3,4,5,6,7,8,9,11],index_col='Date_Time')
dat
#winddata = windat.reset_index().rename(columns={'level_0':'filename'}).drop('level_1',axis=1).set_index('filename')

In [ ]:
data = dat['Canyonlands Research Center'].reset_index().set_index('date_time').drop(columns=['level_0','level_1'],axis=1).dropna(how='all',
                                                                                                                          subset=['windd_sd', 'airt2_avg', 'battv_min', 'precip_tb','windd_avg', 'winds_avg', 'winds_max'])
data = data[['windd_avg','winds_avg']].dropna()
data['windd_avg'] = pd.to_numeric(data['windd_avg'],errors='coerce')
data['winds_avg'] = pd.to_numeric(data['winds_avg'],errors='coerce')

In [ ]:
data = dat[stations.loc[stat,"name"]]

In [ ]:
data.dtypes

In [ ]:
data = data[['windd_avg','winds_avg']].dropna()
data['windd_avg'] = pd.to_numeric(data['windd_avg'],errors='coerce')
data['winds_avg'] = pd.to_numeric(data['winds_avg'],errors='coerce')
ax = WindroseAxes.from_ax()

#wdf = data.rename(columns={'wind_direction_set_1':'direction','wind_speed_set_1':'speed'})
wdf = data.rename(columns={'windd_avg':'direction','winds_avg':'speed'}).dropna()

wdf = wdf[['speed','direction']]
wd = wdf['direction']
ws = wdf['speed']

ax.contour(wd, ws, bins=np.arange(0, 8, 1), cmap=cm.hot, lw=3)
ax.set_legend()

#ax.bar(wd, ws, normed=True, opening=1.0, edgecolor='white')


ax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])

#plt.savefig("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/dugout_wind_rose_map.jpg")
#plt.savefig("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/green_river_wind_rose_map.pdf")

In [ ]:

fig, ax = plt.subplots(figsize=(8, 8), dpi=80)
x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
ax.set_aspect('equal')
#_ = ax.scatter(df['speed_x'], df['speed_y'], alpha=0.25)
wdf['speed_x'] = wdf['speed'] * np.sin(wdf['direction'] * np.pi / 180.0)
wdf['speed_y'] = wdf['speed'] * np.cos(wdf['direction'] * np.pi / 180.0)
ax.scatter(wdf['speed_x'], wdf['speed_y'], alpha=0.01)
Vw = 80
_ = ax.set_xlim([-Vw, Vw])
_ = ax.set_ylim([-Vw, Vw])

In [ ]:
data = {}
hed = {}
for file in glob.glob("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/Data_Downloads/BSF/previous_data/*.csv"):
    dname = os.path.basename(file)
    hed[dname] = pd.read_csv(file,skiprows=1,nrows=8,names=['ind','desc'],index_col='ind').T
    dat = pd.read_csv(file,skiprows=14,na_values=['S','M'])
    data[dname] = dat.rename(columns={'Wind Direction ':'Wind Direction',
                                     'Surface Wind Speed':'Wind Speed',
                                     'Wind Direction Average':'Wind Direction',
                                     'Wind Speed Average':'Wind Speed'}) 
print(data.keys())
windat = pd.concat(data)
hedat = pd.concat(hed)
winddata = windat.reset_index().rename(columns={'level_0':'filename'}).drop('level_1',axis=1).set_index('filename')
hedat = hedat.reset_index().set_index('level_0')

In [ ]:
dat

In [ ]:
    data[statname] = dat.rename(columns={'wind_direction_set_1':'Wind Direction',
                                     'wind_speed_set_1':'Wind Direction'}) 

# MESOWEST Data FILES

In [ ]:
data = {}
hed = {}

for file in glob.glob("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/colorado_river/meso/*.csv"):
    dname = os.path.basename(file)
    
    head_df = pd.read_table(file,skiprows=4,nrows=4,sep=":", names=['ind','desc'])

    head_df['ind'] = head_df['ind'].apply(lambda x: x.replace("# ","").strip(),1)
    head_df = head_df.set_index('ind')
    head_df.columns = ['description']
    statname = head_df.loc['STATION','description'].strip()
    hed[statname] = head_df.T
    dat = pd.read_csv(file,skiprows=[0,1,2,3,4,5,6,7,8,9,11],na_values=['S','M'],parse_dates=True)
    dat['Station_ID'] = dat['Station_ID'].apply(lambda x: x.strip(),1)
    data[statname] = dat.set_index(["Station_ID","Date_Time"])

windat = pd.concat(data).droplevel(0)
windat = windat.rename(columns={'wind_direction_set_1':'Wind Direction',
                                     'wind_speed_set_1':'Wind Speed'}) 
hedat = pd.concat(hed)
#winddata = windat.reset_index().rename(columns={'level_0':'filename'}).drop('level_1',axis=1).set_index('filename')
hedat = hedat.reset_index().set_index('level_0')

hedat['Longitude'] = pd.to_numeric(hedat['LONGITUDE'])
hedat['Latitude'] = pd.to_numeric(hedat['LATITUDE'])

## UCC FILES

In [ ]:
data = {}
hed = {}
for file in glob.glob("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/colorado_river/*.csv"):
    dname = os.path.basename(file)
    hed[dname] = pd.read_csv(file,skiprows=1,nrows=8,names=['ind','desc'],index_col='ind').T
    dat = pd.read_csv(file,skiprows=14,na_values=['S','M'])
    print(hed[dname])
    print(dat.columns)
    data[dname] = dat.rename(columns={'Wind Direction ':'Wind Direction',
                                     'Surface Wind Speed':'Wind Speed',
                                     'Wind Direction Average':'Wind Direction',
                                     'Wind Speed Average':'Wind Speed'}) 
print(data.keys())
windat = pd.concat(data)
hedat = pd.concat(hed)
winddata = windat.reset_index().rename(columns={'level_0':'filename'}).drop('level_1',axis=1).set_index('filename')
hedat = hedat.reset_index().set_index('level_0')

hedat['Longitude'] = pd.to_numeric(hedat['Longitude'])
hedat['Latitude'] = pd.to_numeric(hedat['Latitude'])

In [ ]:
winddata.loc['UCC_ghcn_USC00421163_2022_07_12_1657649854.csv'].columns

In [ ]:
hedat['Longitude'] = pd.to_numeric(hedat['Longitude'])
hedat['Latitude'] = pd.to_numeric(hedat['Latitude'])

In [ ]:
windat

# Plot Windrose Map

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.inset_locator import inset_axes
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

import windrose


minlon, maxlon, minlat, maxlat = (-109.8, -109.1, 37.2, 38.8)

proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(12, 10))
# Draw main ax on top of which we will add windroses
main_ax = fig.add_subplot(1, 1, 1, projection=proj)
main_ax.set_extent([minlon, maxlon, minlat, maxlat], crs=proj)
main_ax.gridlines(draw_labels=True)
main_ax.coastlines()

request = cimgt.OSM()
main_ax.add_image(request, 12)

# Coordinates of the station we were measuring windspeed
wr = {}
for ind in hedat.index.unique():
    df = windat.loc[ind,['Wind Speed','Wind Direction']].dropna()
    ws = df['Wind Speed']
    wd = df['Wind Direction']

    if len(df)>0:
        wr[ind] = inset_axes(main_ax,
                             width=1,
                             height=1,
                             loc='center',
                             bbox_to_anchor=(hedat.loc[ind,'Longitude'], hedat.loc[ind,'Latitude']), # position of the axe
            bbox_transform=main_ax.transData,    # use data coordinate (not axe coordinate)
            axes_class=windrose.WindroseAxes,    # specify the class of the axe
            )

        wr[ind].bar(wd,ws)
        wr[ind].tick_params(labelleft=False, labelbottom=False)
plt.savefig("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/co_wind_rose_map.jpg")
plt.savefig("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/co_wind_rose_map.pdf")

In [ ]:
hedat

In [ ]:
# Coordinates of the station we were measuring windspeed
wr = {}
for ind in hedat.index.unique():
    df = windat.loc[ind,['Wind Speed','Wind Direction']].dropna()
    ws = df['Wind Speed']
    wd = df['Wind Direction']

    ax = WindroseAxes.from_ax()


    ax.contour(wd, ws, bins=np.arange(0, 8, 1), cmap=cm.hot, lw=3)
    ax.set_legend()

    #ax.bar(wd, ws, normed=True, opening=1.0, edgecolor='white')


    ax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])
    
    plt.title(f"{ind} - {hedat.loc[ind,'STATION NAME']}")
    
    plt.savefig(f"G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/{ind}_wind_rose.jpg")
    plt.savefig(f"G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/climate_data/{ind}_wind_rose.pdf")

In [ ]:
hedat.to_csv("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/Weather_Stations.csv")

In [ ]:
axes = WindroseAxes.from_ax()
axes.bar([140]*100, [10]*100, normed=True, opening=0.8, edgecolor='white')
axes.set_legend()
        

In [ ]:
'UCC_awos_KPUC_2020_11_22_1606082818.csv', []
'UCC_scan_2132_2020_11_22_1606082812.csv', 
'UCC_upr_UR309_2020_11_22_1606082826.csv', 
'UCC_utah_dot_UTPSB_2020_11_22_1606082829.csv', 
'UCC_utah_climate_center_UCL18_2020_11_22_1606082822.csv'

In [ ]:
data['UCC_awos_KPUC_2020_11_22_1606082818.csv']['Wind Direction']